# TODO Is 32k aac good enough?

In [ ]:
from notebooks import *
# memory.log.level = 'debug'
sg.init(app=None)

In [ ]:
with contextlib.ExitStack() as stack:
    # stack.enter_context(cache_control(enabled=False))  # Re-runs all of xc page_metadata, which is very slow
    # stack.enter_context(cache_control(refresh=True))  # Re-runs all of xc page_metadata, which is very slow
    recs = (sg.xc_meta
        # .sample(n=1000, random_state=0)
        .sample(n=3, random_state=0)  # TODO TODO XXX Debug rec.id overhaul
        .pipe(recs_featurize_audio_meta)
    )

In [ ]:
with contextlib.ExitStack() as stack:
    # stack.enter_context(cache_control(enabled=False))
    # stack.enter_context(dask_opts(override_use_dask=False))  # XXX Easier-to-read logging [XXX No effect]
    audio_evals = (
        pd.concat([
            # (recs
            (sg.xc_meta

                .sample(random_state=0,
                    n=1,
                    # n=3,
                    # n=10,
                    # n=100,
                )

                # .pipe(tap, lambda df: debug_print('recs_featurize_audio_meta'))
                # XXX Only for our temp usage of sg.xc_meta; disable once we switch back to recs
                .pipe(recs_featurize_audio_meta)

                # .pipe(tap, lambda df: debug_print('load.audio'))
                # Load .audio
                # .pipe(sg.load.audio, scheduler='threads')
                .pipe(sg.load.audio, use=None)

                # Surface .audio_data, for inspection
                .assign(
                    audio_data=lambda df: df.audio.map(lambda x: np.frombuffer(x.unbox._data, dtype=np.uint8)),
                )

                # .pipe(tap, lambda df: debug_print('projection.transform'))
                # Add .feat, .spectro from transcoded .audio
                .pipe(sg.projection.transform)  # .feat
                # .pipe(tap, lambda df: debug_print('features.spectro'))
                .assign(spectro=lambda df: sg.features.spectro(df,  # .spectro is the input for thumb/micro
                    # scheduler='threads',
                    use=None,
                    cache=True,
                ))

                # .pipe(tap, lambda df: debug_print('plot_thumb'))
                # Add .thumb from transcoded .spectro, .audio
                .pipe(df_assign_first, **dict(
                    thumb=df_cell_spectros(plot_thumb.many, sg.features,
                        thumb_s=5,
                        scale=dict(h=int(40 * 4)),  # (scale)
                        # progress=dict(use='dask', scheduler='threads'),
                        progress=dict(use=None),
                        _nocache=True,  # Dev: disable plot_many cache since it's blind to most of our sub-many code changes
                        plot_kwargs=dict(audio_kwargs=dict(
                            **audio_kwargs,  # Override config.audio_kwargs in viz -> display_with_audio
                        )),
                    ),
                ))

                # Drop cols
                #   - .spectro: Trimmed to .micro/.thumb
                #   - .*_stack: Redundant
                .drop(columns=['spectro'])
                [lambda df: [c for c in df if not c.endswith('_stack')]]

                # Cosmetics
                .pipe(df_reorder_cols, first=[
                    'thumb', 'id', 'audio_data', 'duration_s', 'audio',
                ])
                .pipe(df_assign_first,
                    format=audio_kwargs.get('format', ''),
                    bitrate=audio_kwargs.get('bitrate', ''),
                    codec=audio_kwargs.get('codec', ''),
                )

            )
            for audio_kwargs in [

                # TODO TODO Simplify .op().enc(wav) -> .op(), since saving is .enc(wav) by default
                #   - Do:    .op().enc(wav) -> .op()
                #   - Don't: .ext.enc(wav)  -> .ext

                # dict(format='mp3', bitrate='64k'),

                dict(format='wav'),
                dict(format='mp3', bitrate='64k'),
                dict(format='mp4', bitrate='64k',  codec='aac'),
                dict(format='mp4', bitrate='64k',  codec='libfdk_aac'),

                # dict(format='wav'),
                # dict(format='mp3', bitrate='128k'),
                # dict(format='mp3', bitrate='64k'),
                # dict(format='mp3', bitrate='32k'),
                # dict(format='mp3', bitrate='16k'),
                # dict(format='mp3', bitrate='8k'),
                # dict(format='mp4', bitrate='128k', codec='aac'),
                # dict(format='mp4', bitrate='64k',  codec='aac'),
                # dict(format='mp4', bitrate='32k',  codec='aac'),
                # dict(format='mp4', bitrate='128k', codec='libfdk_aac'),
                # dict(format='mp4', bitrate='64k',  codec='libfdk_aac'),
                # dict(format='mp4', bitrate='32k',  codec='libfdk_aac'),
                # dict(format='mp4', bitrate='8k',  codec='libfdk_aac'),

            ]
        ])
    )

INFO     [19:48:01.149] [11431] 813 datasets.xc_meta_to_xc_raw_recs: [2/3 slower] Loading xc.metadata -> xc_raw_recs (.audio, more metadata)...

INFO     [19:48:01.175] [11431] 213 load.audio: in {len(recs): 1}

DEBUG    [19:48:01.178] [11431] 356 load._transcode_path: Read: cache/audio/xc/data/RCKI/157428/audio.mp3.resample(22050,1,16).enc(wav)

INFO     [19:48:01.183] [11431] 231 load.audio: out {len(audio): 1}

DEBUG    [19:48:01.203] [11431] 179 sp14.model.spectro: Features.spectros:in {len(recs): 1, len(recs) per dataset: {xc: 1}, sum(duration_h): 0.00409, sum(samples_mb): 1.24, sum(samples_n): 649728}

...

DEBUG    [19:48:01.947] [11431] 356 load._transcode_path: Read: cache/audio/xc/data/RCKI/157428/audio.mp3.resample(22050,1,16).enc(wav)

INFO     [19:48:01.951] [11431] 231 load.audio: out {len(audio): 1}

DEBUG    [19:48:01.971] [11431] 179 sp14.model.spectro: Features.spectros:in {len(recs): 1, len(recs) per dataset: {xc: 1}, sum(duration_h): 0.00409, sum(samples_mb): 1.24, sum(samples_n): 649728}

DEBUG    [19:48:01.976] [11431] 197 sp14.model.spectro: Features.spectros:out {'(f, sum(t))': [40, 1268]}

DEBUG    [19:48:02.124] [11431] 481 load._transcode_audio: Read: cache/audio/xc/data/RCKI/157428/audio.mp3.resample(22050,1,16).enc(wav).slice(0,5000).enc(mp4,libfdk_aac,64k)

In [ ]:
(audio_evals
    .reset_index()
    .pipe(df_col_map, bitrate=lambda x: x and int(x[:-1]))  # str -> int, for sort
    .sort_values(['xc_id', 'format', 'bitrate'], kind='mergesort')  # Stable sort
    # [lambda df: df.format.isin(['mp3', 'wav'])]
    # [lambda df: df.bitrate.isin([32, ''])]
)